In [1]:
from datasets import load_dataset

ds = load_dataset("cq01/mawps-asdiv-a_svamp")

README.md:   0%|          | 0.00/761 [00:00<?, ?B/s]

(…)-00000-of-00001-52e35a24f615aa7b.parquet:   0%|          | 0.00/527k [00:00<?, ?B/s]

(…)-00000-of-00001-1d9ca8ce89c3de29.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3138 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [2]:
!pip install --upgrade pip

!pip uninstall unsloth -y
!pip uninstall torch torchvision -y
!pip uninstall xformers -y

!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"
!pip install torch==2.4.1 torchvision==0.19.1
!pip install --upgrade xformers==0.0.28.post1

  Created wheel for unsloth: filename=unsloth-2024.11.5-py3-none-any.whl size=161005 sha256=5fd5c76804228bb858b6f651602020aaa24645ead497bcd032f8fdfa3b03829c
  Stored in directory: /tmp/pip-ephem-wheel-cache-z7zaq6ll/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
fastai 2.7.17 requires torchvision>=0.11, which is not installed.
timm 1.0.9 requires torchvision, which is not installe

In [3]:
import os
import torch
import numpy as np
import pandas as pd

In [4]:
for key, value in ds.items():
    print(f"Length of '{key}': {len(value)}")

Length of 'train': 3138
Length of 'validation': 1000


In [5]:
def generate_full_question(example):
    question = example["Question"]
    numbers = example["Numbers"]
    for i, num in enumerate(numbers):
        placeholder = f"number{i}"
        # Check if the number is an integer
        if num.is_integer():
            num_str = str(int(num))
        else:
            num_str = str(num)
        question = question.replace(placeholder, num_str)
    example["Full Question"] = question
    return example

# Apply the transformation to the train and validation sets
ds = ds.map(generate_full_question)

# Now you can access the transformed dataset with the new "Full Question" column
print(ds["train"][23]["Full Question"])

Map:   0%|          | 0/3138 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

A construction company ordered 0.16666666666666666 ton of concrete , 0.16666666666666666 ton of bricks , and 0.5 ton of stone . How many tons of material did the company order in all ?


In [6]:
print(ds["train"][0]["Full Question"])
print(ds["train"][1]["Full Question"])
print(ds["train"][2]["Full Question"])

Bryan took a look at his books as well . If Bryan has 56 books in each of his 9 bookshelves , how many books does he have in total ?
For the fifth grade play , the chairs have been put into 27 rows with 16 chairs in each row . How many chairs have been put out for the play ?
There are 41 short trees and 44 tall trees currently in the park . Park workers will plant 57 short trees today . How many short trees will the park have when the workers are finished ?


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

local_model_path = "unsloth/gemma-2-9b"

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    device_map="auto",  
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from tqdm import tqdm

def extract_number_eval(text):
    """Extract the last number from text using regex."""
    numbers = re.findall(r"[-+]?\d*\.?\d+", text)
    return float(numbers[-1]) if numbers else None

def evaluate_accuracy(model, tokenizer, eval_dataset, num_samples=100):
    """Evaluate model accuracy on a small subset of validation data."""
    model.eval()
    correct = 0
    total = 0
    
    # Sample random indices
    indices = np.random.choice(len(eval_dataset), num_samples, replace=False)
    eval_subset = eval_dataset
    
    evaluation_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

### Input:
{question}

### Response:
"""
    
    results = []
    
    print(f"\nEvaluating accuracy on {num_samples} examples...")
    for example in tqdm(eval_subset):
        try:
            # Prepare input
            prompt = evaluation_prompt.format(question=example["Full Question"])
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            
            # Generate prediction
            with torch.no_grad():
                outputs = model.generate(
                    # **inputs,
                    input_ids=inputs["input_ids"],
                    max_new_tokens=32,
                    use_cache=False,
                    do_sample=False,
                )
            
            # Decode output
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extract prediction and true answer
            predicted_number = extract_number_eval(generated_text)
            true_answer = float(example["Answer"])
            
            # Check if correct
            is_correct = (predicted_number is not None and abs(predicted_number - true_answer) < 1e-6)
            correct += int(is_correct)
            total += 1
            
            # Store result
            results.append({
                "question": example["Full Question"],
                "true_answer": true_answer,
                "predicted": predicted_number,
                "correct": is_correct
            })
            
        except Exception as e:
            print(f"Error processing example: {e}")
            continue
    
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy on {num_samples} samples: {accuracy:.2%}")
    
    return accuracy, results

In [9]:
#### Random test experiment
import re

example = ds["validation"][0]
evaluation_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

### Input:
{question}

### Response:
"""


prompt = evaluation_prompt.format(question=example["Full Question"])

# inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
inputs = tokenizer(
        prompt, 
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=512,  # Match with max_seq_length
).to(model.device)

model.eval()

outputs = model.generate(
    # **inputs,
    **inputs,
    eos_token_id=tokenizer.eos_token_id,  
    max_new_tokens=32,
    use_cache=False,
    do_sample=False,
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
predicted_number = extract_number_eval(generated_text)
print(predicted_number)

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 107

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

### Input:
julia played tag with 18 kids on monday . she played tag with 10 kids on tuesday . how many more kids did she play with on monday than on tuesday ?

### Response:
the answer is 8

8.0


In [12]:
print("\nEvaluating model accuracy before fine-tuning...")
initial_accuracy, initial_results = evaluate_accuracy(
    model, tokenizer, ds["validation"], num_samples=len(ds["validation"])
)

# Save initial results
with open("without_finetuning_initial_evaluation.txt", "w") as f:
    f.write(f"Initial Accuracy: {initial_accuracy:.2%}\n")
    f.write("\nDetailed Results:\n")
    for result in initial_results:  # Save first 10 examples
        f.write(f"\nQ: {result['question']}")
        f.write(f"\nTrue: {result['true_answer']}, Predicted: {result['predicted']}")
        f.write(f"\nCorrect: {result['correct']}\n")

In [11]:
print(initial_accuracy)

0.648
